### An example of a Kalman Filter
##### The below KF takes in data from a .csv named 'data.csv', with the 'time' and 'measurement' columns isolated. These names will need to be modified for use with the Sentinel data. 
##### Adjust the system model, measurement model, and process noise covariance matrix as necessary for your specific application.

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Load data from CSV file
data = pd.read_csv('data.csv')

# Extract time and measurement data from the loaded data
t = data['time'].values
y = data['measurement'].values

# Define the initial state and covariance matrix
x_hat = np.array([0, 0])[:,np.newaxis] # initial guess for the state
P = np.eye(2) # initial guess for the covariance matrix

# Define the system model
dt = t[1] - t[0]
A = np.array([[1, dt], [0, 1]]) # state transition matrix
B = np.array([[dt**2 / 2], [dt]]) # input matrix
u = np.zeros_like(y)[:,np.newaxis] # input (in this case, zero)

# Define the measurement model
H = np.array([[1, 0]]) # observation matrix
R = np.array([[1]]) # measurement noise covariance matrix

# Define the process noise covariance matrix
Q = np.array([[dt**4 / 4, dt**3 / 2], [dt**3 / 2, dt**2]]) * 0.1

# Define arrays to store the filtered data and the Kalman gains
x_hat_filtered = np.zeros_like(x_hat)
K = np.zeros((2, 1, len(t)))

# Apply the Kalman filter to the data
for i in range(len(t)):
    # Predict
    x_hat_minus = A @ x_hat + B @ u[i]
    P_minus = A @ P @ A.T + Q

    # Update
    K[:,:,i] = P_minus @ H.T @ np.linalg.inv(H @ P_minus @ H.T + R)
    x_hat = x_hat_minus + K[:,:,i] @ (y[i] - H @ x_hat_minus)
    P = (np.eye(2) - K[:,:,i] @ H) @ P_minus

    # Store the filtered state estimate
    x_hat_filtered[:,i] = x_hat.ravel()

# Plot the results
plt.plot(t, y, label='Measurement')
plt.plot(t, x_hat_filtered[0,:], label='Filtered')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Value')
plt.show()
